In [1]:
import pandas as pd
import yfinance as yf

In [21]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def get_data(ticker):
    # Define the URL to scrape
    url = f"https://finance.yahoo.com/quote/{ticker}/key-statistics/"

    # Define headers to simulate a browser request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive'
    }

    # Make the request to the webpage
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the page content with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract all tables
        tables = soup.find_all('table')
        
        # List to hold data for CSV
        csv_data = []

        # Process each table
        for i, table in enumerate(tables, start=1):
            # Extract table headers
            headers = [header.get_text(strip=True) for header in table.find_all('th')]
            
            # Extract table rows
            rows = table.find_all('tr')
            for row in rows:
                columns = [col.get_text(strip=True) for col in row.find_all('td')]
                # If the row contains data or headers, add it to the list
                if columns or (i == 1 and headers):
                    csv_data.append(columns)
                    
            # Add headers only if the table has them
            if headers and len(csv_data) > 0:
                csv_data.insert(0, headers)
        
        # Get the current date for the filename
        today = datetime.today().strftime('%Y-%m-%d')
        
        # Convert the data to a DataFrame and save to CSV
        df = pd.DataFrame(csv_data)
        df = df.transpose()
        df.columns = df.iloc[0]
        df = df[1:]
        #display(df)
        rm_cols = ['', None]
        df.drop(columns=rm_cols, inplace=True)
        df = df.iloc[[0]]

        df['Date'] = today
        df['Ticker'] = ticker
        df = df[['Date', 'Ticker'] + [col for col in df.columns if col not in ['Date', 'Ticker']]]
        df.set_index('Date', inplace=True)

        print(f"{ticker} saved")

        return df
    

    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")


In [78]:
import time
tickers = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].str.replace('.', '-').unique().tolist()
first50 = tickers[500:504]
final_df = pd.read_csv('data/new_data.csv', index_col='Date')
for ticker in first50:
    df = get_data(ticker)
    time.sleep(2)
    try:
        final_df = pd.concat([final_df, df])
    except:
        print(f'{ticker} was not added')
final_df.to_csv('data/new_data.csv')

ZBRA saved
ZBH saved
ZTS saved


In [81]:
df = pd.read_csv('data/new_data.csv', index_col='Date')

In [85]:
df[df.Ticker == 'TSLA']

,Ticker,Market Cap,Enterprise Value,Trailing P/E,Forward P/E,PEG Ratio (5yr expected),Price/Sales,Price/Book,Enterprise Value/Revenue,Enterprise Value/EBITDA,...,Shares Short (8/31/2018)4,Short Ratio (8/31/2018)4,Short % of Float (8/31/2018)4,Short % of Shares Outstanding (8/31/2018)4,Shares Short (prior month 7/31/2018)4,Shares Short4,Short Ratio4,Short % of Float4,Short % of Shares Outstanding4,Shares Short (prior month )4
Date,,,,,,,,,,,,,,,,,,,,,
2024-07-23,TSLA,802.11B,785.16B,64.32,101.01,4.56,9.26,12.46,8.29,56.91,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
